In [1]:
import math
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
from sklearn import metrics, datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

random_seed = 30

In [2]:
data = pd.read_csv("../data/featured/data.csv", sep=';')

Xdata = data.drop('teamA_win', axis=1)
ydata = data.teamA_win

In [3]:
Xtrain, Xval, ytrain, yval = train_test_split(Xdata, ydata, test_size=0.4, random_state=random_seed)

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# --- StandardScaler ---
scaler_std = StandardScaler()
Xtrain_std = scaler_std.fit_transform(Xtrain)
Xval_std = scaler_std.transform(Xval)

# --- MinMaxScaler ---
scaler_mm = MinMaxScaler()
Xtrain_mm = scaler_mm.fit_transform(Xtrain)
Xval_mm = scaler_mm.transform(Xval)



In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# === Funkce pro trénink + ladění ===
def tune_logistic_regression(Xtrain, Xval, ytrain, yval, scaler_name):
    print(f"\n=== {scaler_name} ===")
    
    # Hyperparametry k testování
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear', 'saga']
    }
    
    logreg = LogisticRegression(max_iter=5000, random_state=random_seed)
    grid = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(Xtrain, ytrain)
    
    # Nejlepší model
    best_model = grid.best_estimator_
    preds = best_model.predict(Xval)
    acc = accuracy_score(yval, preds)
    
    print("Best params:", grid.best_params_)
    print(f"Validation Accuracy: {acc:.4f}")
    print(classification_report(yval, preds))
    return acc, grid.best_params_

# === StandardScaler ===
scaler_std = StandardScaler()
Xtrain_std = scaler_std.fit_transform(Xtrain)
Xval_std   = scaler_std.transform(Xval)

# === MinMaxScaler ===
scaler_mm = MinMaxScaler()
Xtrain_mm = scaler_mm.fit_transform(Xtrain)
Xval_mm   = scaler_mm.transform(Xval)

# === Spuštění ladění ===
acc_std, params_std = tune_logistic_regression(Xtrain_std, Xval_std, ytrain, yval, "StandardScaler")
acc_mm,  params_mm  = tune_logistic_regression(Xtrain_mm,  Xval_mm,  ytrain, yval, "MinMaxScaler")

# === Porovnání ===
print("\n=== Summary ===")
print(f"StandardScaler: {acc_std:.4f} (params: {params_std})")
print(f"MinMaxScaler:   {acc_mm:.4f} (params: {params_mm})")



=== StandardScaler ===
Best params: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
Validation Accuracy: 0.7481
              precision    recall  f1-score   support

           0       0.73      0.67      0.70       291
           1       0.76      0.81      0.78       372

    accuracy                           0.75       663
   macro avg       0.75      0.74      0.74       663
weighted avg       0.75      0.75      0.75       663


=== MinMaxScaler ===
Best params: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Validation Accuracy: 0.7496
              precision    recall  f1-score   support

           0       0.73      0.69      0.71       291
           1       0.77      0.80      0.78       372

    accuracy                           0.75       663
   macro avg       0.75      0.74      0.74       663
weighted avg       0.75      0.75      0.75       663


=== Summary ===
StandardScaler: 0.7481 (params: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'})
MinMaxScaler:  

In [9]:
clfLOG = LogisticRegression(solver='newton-cg')
clfLOG.fit(Xtrain,ytrain)

print('\nValidační přesnost:', clfLOG.score(Xval,yval))


Validační přesnost: 0.7481146304675717
